Adheres to the logic of `process_data.py`, processing `yearly_dataset.csv`.
Run in a setting with `output/yearly_dataset.csv` (output of `retrieve_data.py`).

In [1]:
import pandas as pd
import numpy as np
import warnings

# Suppress warnings for uncluttered output.
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

INPUT_CSV = "output/yearly_dataset.csv"
SPY_CSV = "output/spy_data.csv"
OUTPUT_CSV = "output/processed_dataset.csv"
OUTPUT_2024_CSV = "output/processed_2024_dataset.csv"
EXCLUDED_COLS = ["year", "ticker"]
PRICE_COL = ["price_change"]

# Print missing value %.
def log_missing_pct(df, step_name):
    if df.empty:
        print(f"{step_name}: DataFrame is empty, missing %: 100%")
        return
    total_cells = df.size
    missing_cells = df.isna().sum().sum()
    missing_pct = round(missing_cells / total_cells * 100, 2) if total_cells > 0 else 100.0
    print(f"{step_name}: missing %: {missing_pct}%")
    
    missing_summary = df.isna().sum() / len(df) * 100
    missing_summary = missing_summary[missing_summary > 0].round(2)
    if not missing_summary.empty:
        print("\nPer-column missing % (non-zero only):")
        print(missing_summary)
    else:
        print("\nNo missing values per column.")
    print("\n" + "="*50 + "\n")

df = pd.read_csv(INPUT_CSV).copy()
print(f"DataFrame shape: {df.shape} (rows, columns)")
log_missing_pct(df, "After loading")

DataFrame shape: (9588, 384) (rows, columns)
After loading: missing %: 9.09%

Per-column missing % (non-zero only):
price_begin                                                   3.39
price_end                                                     3.39
price_change                                                  3.39
income_statement_reportedCurrency                           100.00
income_statement_cik                                          1.24
                                                             ...  
financial_growth_tenYBottomLineNetIncomeGrowthPerShare       38.84
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare      38.84
financial_growth_threeYBottomLineNetIncomeGrowthPerShare     38.84
financial_growth_fetched_at_utc                             100.00
financial_growth_calendarYear                                 1.22
Length: 382, dtype: float64




In [2]:
# Delete price_begin and price_end columns
df = df.drop(columns=["price_begin", "price_end"], errors="ignore")
log_missing_pct(df, "After dropping price_begin/end")

After dropping price_begin/end: missing %: 9.12%

Per-column missing % (non-zero only):
price_change                                                  3.39
income_statement_reportedCurrency                           100.00
income_statement_cik                                          1.24
income_statement_filingDate                                 100.00
income_statement_acceptedDate                               100.00
                                                             ...  
financial_growth_tenYBottomLineNetIncomeGrowthPerShare       38.84
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare      38.84
financial_growth_threeYBottomLineNetIncomeGrowthPerShare     38.84
financial_growth_fetched_at_utc                             100.00
financial_growth_calendarYear                                 1.22
Length: 380, dtype: float64




In [3]:
# Save index data and remove from main dataset
spy_df = df[df["ticker"] == "SPY"][["year", "price_change"]]
spy_df[["year", "price_change"]].to_csv(SPY_CSV, index=False)
df = df[df["ticker"] != "SPY"]
log_missing_pct(df, "After removing SPY")

After removing SPY: missing %: 8.93%

Per-column missing % (non-zero only):
price_change                                                  3.40
income_statement_reportedCurrency                           100.00
income_statement_cik                                          1.03
income_statement_filingDate                                 100.00
income_statement_acceptedDate                               100.00
                                                             ...  
financial_growth_tenYBottomLineNetIncomeGrowthPerShare       38.71
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare      38.71
financial_growth_threeYBottomLineNetIncomeGrowthPerShare     38.71
financial_growth_fetched_at_utc                             100.00
financial_growth_calendarYear                                 1.01
Length: 380, dtype: float64




In [4]:
# Delete ticker+year combinations if missing annual return
df = df[df["price_change"].notna()]
log_missing_pct(df, "After removing ticker+year combinations if missing price_change")

After removing ticker+year combinations if missing price_change: missing %: 8.77%

Per-column missing % (non-zero only):
income_statement_reportedCurrency                           100.00
income_statement_cik                                          1.03
income_statement_filingDate                                 100.00
income_statement_acceptedDate                               100.00
income_statement_fiscalYear                                   1.03
                                                             ...  
financial_growth_tenYBottomLineNetIncomeGrowthPerShare       39.06
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare      39.06
financial_growth_threeYBottomLineNetIncomeGrowthPerShare     39.06
financial_growth_fetched_at_utc                             100.00
financial_growth_calendarYear                                 1.02
Length: 379, dtype: float64




In [5]:
# Delete feature columns for specific years if >50% missing values
feature_cols = [c for c in df.columns if c not in EXCLUDED_COLS + PRICE_COL]
for year in df["year"].unique():
    year_mask = df["year"] == year
    if not df[year_mask].empty:
        for col in feature_cols[:]:
            if col in df.columns:
                missing_rate = df.loc[year_mask, col].isna().mean()
                if missing_rate > 0.5 or df.loc[year_mask, col].isna().all():
                    # Exclude the feature for this year by setting to NaN (proxy for deletion)
                    df.loc[year_mask, col] = np.nan
                    # If the entire column is now NaN across all years, drop it
                    if df[col].isna().all():
                        df = df.drop(columns=[col])
                        feature_cols.remove(col)
log_missing_pct(df, "After removing feature+year combos")

After removing feature+year combos: missing %: 1.55%

Per-column missing % (non-zero only):
income_statement_cik                                         1.03
income_statement_fiscalYear                                  1.03
income_statement_revenue                                     1.03
income_statement_costOfRevenue                               1.03
income_statement_grossProfit                                 1.03
                                                            ...  
financial_growth_growthCapitalExpenditure                   39.06
financial_growth_tenYBottomLineNetIncomeGrowthPerShare      39.06
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare     39.06
financial_growth_threeYBottomLineNetIncomeGrowthPerShare    39.06
financial_growth_calendarYear                                1.02
Length: 351, dtype: float64




In [6]:
# Delete ticker+year combinations if >50% are missing
for (ticker, year), group in df.groupby(["ticker", "year"]):
    if not group.empty:
        if group[feature_cols].isna().mean().mean() > 0.5:
            df = df[~((df["ticker"] == ticker) & (df["year"] == year))]
log_missing_pct(df, "After removing ticker+year combos")

# Check if >5% missing remains (pause logic adapted for notebook)
total_cells = df.size
missing_cells = df.isna().sum().sum()
missing_pct = round(missing_cells / total_cells * 100, 2) if total_cells > 0 else 100.0
if missing_pct >= 5:
    print(f"After deletion missing %: {missing_pct}% >= 5%; review data before continuing.")
else:
    print(f"After deletion missing %: {missing_pct}% < 5%; continuing.")

After removing ticker+year combos: missing %: 0.57%

Per-column missing % (non-zero only):
income_statement_cik                                         0.01
income_statement_fiscalYear                                  0.01
income_statement_revenue                                     0.01
income_statement_costOfRevenue                               0.01
income_statement_grossProfit                                 0.01
                                                            ...  
financial_growth_growthCapitalExpenditure                   38.44
financial_growth_tenYBottomLineNetIncomeGrowthPerShare      38.44
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare     38.44
financial_growth_threeYBottomLineNetIncomeGrowthPerShare    38.44
financial_growth_calendarYear                                0.01
Length: 351, dtype: float64


After deletion missing %: 0.57% < 5%; continuing.


In [7]:
# Fill missing values with per-ticker median across all years for that feature
for ticker in df["ticker"].unique():
    ticker_mask = df["ticker"] == ticker
    for col in feature_cols:
        if col in df.columns:
            median_val = df.loc[ticker_mask, col].median()
            df.loc[ticker_mask & df[col].isna(), col] = median_val
log_missing_pct(df, "After per-ticker median fill")

After per-ticker median fill: missing %: 0.54%

Per-column missing % (non-zero only):
financial_growth_ebitdaGrowth                               38.43
financial_growth_growthCapitalExpenditure                   38.43
financial_growth_tenYBottomLineNetIncomeGrowthPerShare      38.43
financial_growth_fiveYBottomLineNetIncomeGrowthPerShare     38.43
financial_growth_threeYBottomLineNetIncomeGrowthPerShare    38.43
dtype: float64




In [8]:
# Fill remaining missing values with global median for that feature in that year
for year in df["year"].unique():
    year_mask = df["year"] == year
    if not df[year_mask].empty:
        for col in feature_cols:
            if col in df.columns:
                global_median = df.loc[year_mask, col].median()
                df.loc[year_mask & df[col].isna(), col] = global_median
log_missing_pct(df, "After global median fill")

After global median fill: missing %: 0.0%

No missing values per column.




In [9]:
# Delete ticker+year combos with >10% extreme z-scores
all_cols = [c for c in df.columns if c not in EXCLUDED_COLS]
deleted_ticker_years = []
for (ticker, year), group in df.groupby(["ticker", "year"]):
    if not group.empty:
        extreme_count = 0
        year_mask = df["year"] == year
        for col in all_cols:
            if col in group.columns and group[col].dtype in ["float64", "int64"]:
                # Use year-specific mean and std
                mean_val = df.loc[year_mask, col].mean()
                std_val = df.loc[year_mask, col].std()
                if std_val > 0:
                    z_score = (group[col].iloc[0] - mean_val) / std_val
                    if abs(z_score) > 5:
                        extreme_count += 1
        extreme_pct = (extreme_count / len(all_cols)) * 100 if all_cols else 0
        if extreme_pct > 10:
            deleted_ticker_years.append((ticker, year))
            df = df[~((df["ticker"] == ticker) & (df["year"] == year))]
print(f"Deleted {len(deleted_ticker_years)} ticker-year combos due to extremes.")
log_missing_pct(df, "After removing extreme z-scores")

Deleted 94 ticker-year combos due to extremes.
After removing extreme z-scores: missing %: 0.0%

No missing values per column.




In [10]:
# Cap remaining extreme values to within z-scores of ±5
for col in all_cols:
    if col in df.columns and df[col].dtype in ["float64", "int64"]:
        mean_val = df[col].mean()
        std_val = df[col].std()
        if std_val > 0:
            z_scores = (df[col] - mean_val) / std_val
            valid_values = df[col][z_scores.abs() <= 5]
            max_val = valid_values.max() if not valid_values.empty else df[col].max()
            min_val = valid_values.min() if not valid_values.empty else df[col].min()
            df.loc[(z_scores > 5) & (df[col].notna()), col] = max_val
            df.loc[(z_scores < -5) & (df[col].notna()), col] = min_val
log_missing_pct(df, "After capping extreme values")

After capping extreme values: missing %: 0.0%

No missing values per column.




In [11]:
    print("Adding label column beat_index")
    df["beat_index"] = np.nan
    spy_prices = pd.read_csv(SPY_CSV).set_index("year")["price_change"]
    for year in df["year"].unique():
        if year < 2024:
            next_year = year + 1
            if next_year in spy_prices.index:
                spy_change = spy_prices[next_year]
                year_mask = df["year"] == year
                df.loc[year_mask, "beat_index"] = (df.loc[year_mask, "price_change"] > spy_change).astype(int)

    log_missing_pct(df, "After adding beat_index")
    pre2024 = df[df["year"] < 2024]
    if not pre2024.empty:
        if pre2024["beat_index"].isna().sum() == 0:
            print("beat_index has no missing values prior to 2024")
        else:
            pct = 100 * pre2024["beat_index"].isna().mean()
            print(f"beat_index missing {pct:.2f}% prior to 2024")

    deleted_ticker_years = []
    for (ticker, year), group in df.groupby(["ticker", "year"]):
        # Drop entire combo only if ALL rows are unlabeled for that past year
        if year < 2024 and group["beat_index"].isna().all():
            deleted_ticker_years.append((ticker, year))
            df = df[~((df["ticker"] == ticker) & (df["year"] == year))]

    print(f"Deleted {len(deleted_ticker_years)} unlabeled ticker-year combos.")
    log_missing_pct(df, "After dropping unlabeled combos")
    pre2024 = df[df["year"] < 2024]
    if not pre2024.empty:
        if pre2024["beat_index"].isna().sum() == 0:
            print("beat_index has no missing values prior to 2024")
        else:
            pct = 100 * pre2024["beat_index"].isna().mean()
            print(f"beat_index missing {pct:.2f}% prior to 2024")

Adding label column beat_index
After adding beat_index: missing %: 0.02%

Per-column missing % (non-zero only):
beat_index    5.48
dtype: float64


beat_index has no missing values prior to 2024
Deleted 0 unlabeled ticker-year combos.
After dropping unlabeled combos: missing %: 0.02%

Per-column missing % (non-zero only):
beat_index    5.48
dtype: float64


beat_index has no missing values prior to 2024


In [12]:
# Delete price_change column
df = df.drop(columns=PRICE_COL, errors="ignore")
log_missing_pct(df, "After removing price_change")

After removing price_change: missing %: 0.02%

Per-column missing % (non-zero only):
beat_index    5.48
dtype: float64




In [13]:
# Split dataset into 2005-2023 (main) and 2024 (final prediction)
df_main = df[df["year"].between(2005, 2023)]
df_2024 = df[df["year"] == 2024]
log_missing_pct(df_main, "2005-2023 dataset")
log_missing_pct(df_2024, "2024 dataset")

# Check for any remaining missing values
missing_found = False
for df_subset, name in [(df_main, "2005-2023"), (df_2024, "2024")]:
    for col in df_subset.columns:
        if col == "beat_index" and name == "2024":
            continue  # Allow NaN in beat_index for 2024
        missing_rows = df_subset[df_subset[col].isna()][["ticker", "year"]]
        for _, row in missing_rows.iterrows():
            print(f"Missing value in {col} for {row['ticker']} in {row['year']} ({name})")
            missing_found = True
if not missing_found:
    print("No missing values found.")

# Save processed datasets
df_main.to_csv(OUTPUT_CSV, index=False)
df_2024.to_csv(OUTPUT_2024_CSV, index=False)
print(f"Saved processed dataset to {OUTPUT_CSV} (shape: {df_main.shape})")
print(f"Saved 2024 dataset to {OUTPUT_2024_CSV} (shape: {df_2024.shape})")

2005-2023 dataset: missing %: 0.0%

No missing values per column.


2024 dataset: missing %: 0.28%

Per-column missing % (non-zero only):
beat_index    100.0
dtype: float64


No missing values found.
Saved processed dataset to output/processed_dataset.csv (shape: (8559, 354))
Saved 2024 dataset to output/processed_2024_dataset.csv (shape: (496, 354))


- Removes outliers, splits dataset, fills missings, and adds labels.
- Outputs: `processed_dataset.csv` and `processed_2024_dataset.csv`.